In [24]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/AkshitaTomar1/WineQualityPrediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="AkshitaTomar1"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6961ae0d59a6b2f20fb13fa171a9a0e2b2b44f7b"

In [43]:
import os
os.chdir("C:\\Users\\VISHU\\Desktop\\End-to-EndProjects\\WineQualityPrediction")
%pwd

'C:\\Users\\VISHU\\Desktop\\End-to-EndProjects\\WineQualityPrediction'

In [44]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [45]:
from src.WineQualityPrediction.constants import *
from src.WineQualityPrediction.utils.common import read_yaml,create_directories,save_json

In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/AkshitaTomar1/WineQualityPrediction.mlflow"
           
        )

        return model_evaluation_config


In [47]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import numpy as np
import joblib

In [48]:
import mlflow.models.signature
from mlflow.models.signature import infer_signature

In [49]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def log_into_mlflow(self):

        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities=model.predict(test_x)

            (rmse, mae, r2)=self.eval_metrics(test_y,predicted_qualities)

            #saving metrics as local
            scores={"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("r2",r2)
            mlflow.log_metric("mae",mae)
            
            input_example = test_x.head(1)
            signature = infer_signature(test_x, predicted_qualities)

            #model registry does not work with the file store
            if tracking_url_type_store != "file": 
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticnetModel",signature=signature,
                input_example=input_example)
            else:
                mlflow.sklearn.log_model(model,"model")






    

In [50]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-17 20:57:26,904: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-17 20:57:26,920: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-17 20:57:26,946: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-17 20:57:26,961: INFO: common: created directory at: artifacts]
[2025-01-17 20:57:26,963: INFO: common: created directory at: artifacts/model_evaluation]


[2025-01-17 20:57:28,939: INFO: common: json file saved at :artifacts\model_evaluation\metrics.json]


c:\Users\VISHU\Desktop\End-to-EndProjects\WineQualityPrediction\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Successfully registered model 'ElasticnetModel'.
2025/01/17 20:57:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run shivering-bug-645 at: https://dagshub.com/AkshitaTomar1/WineQualityPrediction.mlflow/#/experiments/0/runs/aae607bcac0944a2b5bcb717a62ea0c0
🧪 View experiment at: https://dagshub.com/AkshitaTomar1/WineQualityPrediction.mlflow/#/experiments/0
